# Summary

> **Notes:**
>
> - Compress the files back after extracting relevant information!!!
> - Use [`shutil.make_archive`](https://docs.python.org/3/library/shutil.html#shutil.make_archive) to compress files as you download them!

# Imports

In [1]:
%run _imports.ipynb

Setting the PYTHON_VERSION environment variable.
Setting the SPARK_MASTER environment variable.
Setting the DB_TYPE environment variable.
Setting the DB_PORT environment variable.


2017-12-06 10:00:58.119047


In [2]:
NOTEBOOK_NAME = 'download'
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Download

## OA package

In [3]:
urllib.request.urlretrieve(
    'ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_file_list.csv',
    f'{NOTEBOOK_NAME}/oa_file_list.csv'
)

('download/oa_file_list.csv', <email.message.Message at 0x2b0190e68f98>)

In [4]:
oa_file_list = pd.read_csv(f'{NOTEBOOK_NAME}/oa_file_list.csv')

In [5]:
oa_file_list.head()

,File,Article Citation,Accession ID,Last Updated (YYYY-MM-DD HH:MM:SS),PMID,License
0,oa_package/08/e0/PMC13900.tar.gz,Breast Cancer Res. 2001 Nov 2; 3(1):55-60,PMC13900,2017-04-26 12:15:50,11250746.0,NO-CC CODE
1,oa_package/b0/ac/PMC13901.tar.gz,Breast Cancer Res. 2001 Nov 9; 3(1):61-65,PMC13901,2016-01-20 10:58:46,11250747.0,NO-CC CODE
2,oa_package/f7/98/PMC13902.tar.gz,Breast Cancer Res. 2001 Nov 8; 3(1):66-75,PMC13902,2006-02-02 19:37:52,11250748.0,NO-CC CODE
3,oa_package/9c/7f/PMC13911.tar.gz,Breast Cancer Res. 2000 Nov 16; 2(1):59-63,PMC13911,2013-03-17 14:00:52,11056684.0,NO-CC CODE
4,oa_package/c6/fb/PMC13912.tar.gz,Breast Cancer Res. 2000 Dec 6; 2(1):64-72,PMC13912,2013-03-17 14:00:52,11400682.0,NO-CC CODE


In [6]:
oa_package_df = oa_file_list[['File', 'Accession ID']].drop_duplicates()
oa_package_df.columns = ['file_path', 'pmc_id']

In [7]:
oa_package_df.head()

,file_path,pmc_id
0,oa_package/08/e0/PMC13900.tar.gz,PMC13900
1,oa_package/b0/ac/PMC13901.tar.gz,PMC13901
2,oa_package/f7/98/PMC13902.tar.gz,PMC13902
3,oa_package/9c/7f/PMC13911.tar.gz,PMC13911
4,oa_package/c6/fb/PMC13912.tar.gz,PMC13912


## Functions

In [8]:
ARCHIVE_DIR = f"{NOTEBOOK_NAME}/pmc_3"
os.makedirs(ARCHIVE_DIR, exist_ok=True)

In [9]:
EXTENSIONS_TO_KEEP = ['.nxml', '.xml', '.csv', '.csv.gz', '.tsv', '.tsv.gz', '.xls', '.xlsx']

In [23]:
import datetime
import tarfile
from typing import List
import zipfile

import dateutil.parser


def extract_tar_file(filename, output_dir) -> List[str]:
    with tarfile.open(filename, 'r:gz') as tar_file:
        members = [
            m
            for m in tar_file.getmembers()
            if any(m.name.endswith(suffix) for suffix in EXTENSIONS_TO_KEEP)
        ]
        tar_file.extractall(path=output_dir, members=members)
    return [m.name for m in members]
    
    
def extract_zip_file(filename, output_dir) -> List[str]:
    with zipfile.ZipFile(filename) as zip_file:
        members = [
            m
            for m in zip_file.namelist()
            if any(m.endswith(suffix) for suffix in EXTENSIONS_TO_KEEP)
        ]
        zip_file.extractall(path=output_dir, members=members)
    return members    


def download(pmc_id, source_url, output_dir, overwrite=False):
    import socket
    socket.setdefaulttimeout(60)

    info = {
        'pmc_id': pmc_id,
        'source_url': source_url,
        'output_dir': output_dir,
    }

    output_dir = op.join(ARCHIVE_DIR, output_dir)
    
    if output_dir.strip('/').endswith(pmc_id):
        info_file = op.join(output_dir, 'info.json')
    else:
        info_file = op.join(output_dir, pmc_id, 'info.json')
    
    info['info_file'] = op.relpath(info_file, ARCHIVE_DIR)

    if not overwrite and op.isfile(info_file):
        info.update({'error': "Info file already exists!"})
        return info

    with tempfile.NamedTemporaryFile(dir='/dev/shm') as tmp_file, \
            tempfile.:
        urllib.request.urlretrieve(source_url, tmp_file.name)
        if source_url.endswith('.tar.gz'):
            extracted_files = extract_tar_file(tmp_file.name, output_dir)
        elif source_url.endswith('.zip'):
            extracted_files = extract_zip_file(tmp_file.name, output_dir)
        else:
            raise Exception(f"Unsupported archive {source_url}!")
        
        info.update({
            'source_url': source_url,
            'created_on': datetime.datetime.now().isoformat(),
            'extracted_files': extracted_files,
        })
        os.makedirs(op.dirname(info_file), exist_ok=True)
        with open(info_file, 'wt') as fout:
            json.dump(info, fout)
    
    return info

SyntaxError: invalid syntax (<ipython-input-23-9d77247aa768>, line 61)

In [11]:
pmc_id = 'PMC1125901'
source_url = 'ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/NON-OA/PMC1125900-PMC1129899/PMC1125901.zip'
output_dir = 'suppl/NON-OA/PMC1125900-PMC1129899/PMC1125901'

download(pmc_id, source_url, output_dir, True)

{'created_on': '2017-12-06T10:01:11.360791',
 'extracted_files': [],
 'info_file': 'suppl/NON-OA/PMC1125900-PMC1129899/PMC1125901/info.json',
 'output_dir': 'suppl/NON-OA/PMC1125900-PMC1129899/PMC1125901',
 'pmc_id': 'PMC1125901',
 'source_url': 'ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/NON-OA/PMC1125900-PMC1129899/PMC1125901.zip'}

## Downlaod files

In [12]:
ARCHIVE_DIR

'download/pmc_3'

In [13]:
to_download = pd.read_csv('pmc_statistics/to_download.csv')

In [14]:
to_download.head()

,pmc_id,source_url,output_dir
0,PMC13900,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/08/e0/PMC13900.tar.gz,oa_package/08/e0/
1,PMC13901,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/b0/ac/PMC13901.tar.gz,oa_package/b0/ac/
2,PMC13902,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/f7/98/PMC13902.tar.gz,oa_package/f7/98/
3,PMC13911,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/9c/7f/PMC13911.tar.gz,oa_package/9c/7f/
4,PMC13912,ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/c6/fb/PMC13912.tar.gz,oa_package/c6/fb/


In [15]:
to_download.tail()

,pmc_id,source_url,output_dir
2083565,PPR7031,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/OA/preprint/PPR7031.zip,suppl/OA/preprint/PPR7031
2083566,PPR7032,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/OA/preprint/PPR7032.zip,suppl/OA/preprint/PPR7032
2083567,PPR7033,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/OA/preprint/PPR7033.zip,suppl/OA/preprint/PPR7033
2083568,PPR7034,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/OA/preprint/PPR7034.zip,suppl/OA/preprint/PPR7034
2083569,PPR7035,ftp://ftp.ebi.ac.uk/pub/databases/pmc/suppl/OA/preprint/PPR7035.zip,suppl/OA/preprint/PPR7035


In [16]:
# info_list = []

# for row in tqdm.tqdm_notebook(to_download.itertuples(), total=len(to_download)):
#     info = download(row.pmc_id, row.source_url, row.output_dir, overwrite=False)
#     info_list.append(info)

In [17]:
chunk_size = 50_000
finished_chunks = []

In [18]:
len(to_download) // chunk_size

41

In [19]:
finished_futures = []
for i in range(9, 10):
    chunk_offset = chunk_size * i
    if i in finished_chunks:
        continue
    with concurrent.futures.ThreadPoolExecutor(8) as pool:
        futures = [
            pool.submit(download, row.pmc_id, row.source_url, row.output_dir, overwrite=False) 
            for row in itertools.islice(to_download.itertuples(), chunk_offset, chunk_offset + chunk_size)
        ]
        for future in tqdm.tqdm_notebook(
                concurrent.futures.as_completed(futures), 
                total=len(futures),
                desc=f"Chunk {i}"):
            finished_futures.append(future)

A Jupyter Widget

In [20]:
# 2819

In [21]:
# results = []
# for i in range(4, 10):
#     chunk_offset = chunk_size * i
#     if i in finished_chunks:
#         continue
#     for row in tqdm.tqdm_notebook(
#             itertools.islice(to_download.itertuples(), chunk_offset, chunk_offset + chunk_size),
#             total=chunk_size,
#             desc=f"Chunk {i}"):
#         try:
#             result = download(row.pmc_id, row.source_url, row.output_dir, overwrite=False)
#         except Exception as e:
#             result = row._asdict()
#             result['error'] = str(e)
#         results.append(result)

In [22]:
# 2025